In [ ]:
#The prerequiste version of google chrome driver in path 
chromeOptions = webdriver.ChromeOptions()
chromeOptions.binary_location = "C:\Program Files\Google\Chrome\Application\chrome.exe"
b = webdriver.Chrome(options=chromeOptions, executable_path="C:\Users\dhananjayan\Downloads\chromedriver_win32\chromedriver.exe\chrome.exe")

In [19]:
#import required libraries
from bs4 import BeautifulSoup
import re 
from random import randint
from time import sleep
import pandas as pd
from selenium import webdriver
browser = webdriver.Chrome('chromedriver_win32/chromedriver.exe')

In [20]:
browser.get('https://www.linkedin.com/login?fromSignIn=true&trk=guest_homepage-basic_nav-header-signin')


#Enter login info:
elementID = browser.find_element_by_id('username')
elementID.send_keys('xxxxxxxxxxxxx')

elementID = browser.find_element_by_id('password')
elementID.send_keys(xxxxxxxxxxxxxx)
#Note: replace the keys "username" and "password" with your LinkedIn login info
elementID.submit()
sleep(3)

In [21]:
#there is a default message notification pop is set as default in linkedin, so , I just initiate a click to pushdown notification

from selenium.common.exceptions import NoSuchElementException
try:
    if browser.find_element_by_class_name('msg-overlay-list-bubble--is-minimized') is not None:
        pass
except NoSuchElementException:
    try:
        if browser.find_element_by_class_name('msg-overlay-bubble-header') is not None:
            browser.find_element_by_class_name('msg-overlay-bubble-header').click()
    except NoSuchElementException:
        pass

In [22]:
#Go to webpage job search and filter as per requirements (here just jobs posted past 24 hours) automated
browser.get('https://www.linkedin.com/jobs/?showJobAlertsModal=false')

#Find search box
jobID = browser.find_element_by_class_name('jobs-search-box__text-input')
search = browser.find_element_by_class_name('jobs-search-box__submit-button')
search.click()
sleep(5)

browser.find_element_by_xpath('//button[text()="Date Posted"]').click()
sleep(3)
browser.find_element_by_xpath('//span[text()="Past 24 hours"]').click()
sleep(3)
browser.find_element_by_xpath('//span[text()="Show results"]').click()
sleep(3)

In [23]:
#getting the pagesource and reading the source

pageSource = browser.page_source
soup = BeautifulSoup(pageSource, 'lxml')

# searching for all job containers
location_containers = soup.find_all('div', class_ = 'job-card-container')

#Initialze variables to store scarped data
job_id = []
post_title = []
company_name = []
job_location = []
job_desc = []

for job in location_containers:
    
    # job title
    job_titles = job.find(class_="job-card-list__title").text
    post_title.append(job_titles)
    
    job_ids = job.find('a',href=True)['href']
    job_ids = re.search("[0-9]+",job_ids)[0]
    job_id.append(job_ids)
    
    company_names = job.select_one('img')['alt']
    company_name.append(company_names[0:-4])
    
    job_locations = job.find("li", class_="job-card-container__metadata-item").text
    job_location.append(job_locations)

#the tricky part is to scarpe job description which is shown after clicking the respective hover_job_tiles
for x in range(1,len(job_id)+1):
    job_xpath = '//section[1]/div/div/ul/li[{}]'.format(x)
    browser.find_element_by_xpath(job_xpath).click()
    sleep(3)
    jobdesc_xpath = '//section[2]/div/div/div[1]/div/div[2]/article/div/div[1]/span'
    job_descs = browser.find_element_by_xpath(jobdesc_xpath).text
    job_desc.append(job_descs)

In [25]:
#we can store the data in a pandas dataframe input as keyvalue pair,
job_data = pd.DataFrame({'Job ID': job_id,
'Company Name': company_name,
'Post': post_title,
'Location': job_location,
'Description': job_desc,                         
})

#string manipulation can be done after this
job_data['Post'] = job_data['Post'].str.replace('\n',' ')
job_data['Description']=job_data['Description'].str.replace('\n',' ')


In [26]:
#created database
job_data

,Job ID,Company Name,Post,Location,Description
0,2465788985,Amazon,Digital Content Associate,"Chennai, Tamil Nadu, India",Filing of applications and complying other rel...
1,2474142681,Crossover for Work,Technical Team Lead (Remote) - $1...,"Mumbai, Maharashtra, India",Are you a great developer whose skills have be...
2,2477313583,Cognizant,Jr. Executive - TAG,"Chennai, Tamil Nadu, India",#Coordinator – North America Campus Recruitmen...
3,2466400546,Accenture in India,Associate-Reporting,"Chennai, Tamil Nadu, India",Skill required: Reporting - Microsoft Power BI...
4,2465749927,Philips,Data Specialist,"Chennai, Tamil Nadu, India",Responsibilities Ability to understand the dat...
5,2454857799,Blackboard,Data Analyst,"Chennai, Tamil Nadu, India","Data Analyst Chennai, India The Opportunity ..."
6,2477469508,RemoteHub,WORK FROM HOME WITHOUT STRESS ...,"Madanpur, Bihar, India",Hello My Name is Ruth Clara I work for Orphans...


In [27]:
#we can store the data in any database Create an in-memory SQLite database, when In production you can config to any database

from sqlalchemy import create_engine
engine = create_engine('sqlite://', echo=False)

In [28]:
#dumping the data
job_data.to_sql('job_datas', con=engine)

In [29]:
print(engine.execute("SELECT * FROM job_datas").fetchall())

[(0, '2465788985', 'Amazon ', '             Digital Content Associate           ', 'Chennai, Tamil Nadu, India', 'Filing of applications and complying other relevant procedures for registration of Trade Marks, Designs and Copyrights.2. Attending matters before Tr ... (33 characters truncated) ... IPAB etc. at Chennai as applicable.  Junior Associate  Skills: Associate, Legal  Experience: 2.00-3.00 Years  Education: Bachelors of Law (B.L/L.L.B)'), (1, '2474142681', 'Crossover for Work ', '             Technical Team Lead (Remote) - $100,000/year USD           ', 'Mumbai, Maharashtra, India', 'Are you a great developer whose skills have been noticed by both your manager and peers alike? Your teammates come to you for technical advice and in ... (4444 characters truncated) ... tps://www.crossover.com/auth/reset-password if you already applied using LinkedIn EasyApply. Crossover Job Code: LJ-3879-IN-Mumbai-TechnicalTeamL.010'), (2, '2477313583', 'Cognizant ', '             Jr. Executive - 